In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# make the necessary imports 
import torch
from torch.utils.data import Dataset, random_split
from torchvision import datasets,models
import torchvision
import torch.nn as nn
from torchvision.transforms import ToTensor,Compose,RandomCrop,Resize,ToPILImage,RandomResizedCrop,RandomHorizontalFlip,Normalize,ConvertImageDtype
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import DataLoader

import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# to add - transforms - resized to 256,256
# generate dataset class for our custom dataset
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=Compose([Resize((256,256)),RandomResizedCrop(224),ConvertImageDtype(dtype=torch.float),Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]), target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        sample = {"image": image, "label": label}
        return image,label


In [ ]:
dataset=CustomImageDataset('../input/cassava-leaf-disease-classification/train.csv','../input/cassava-leaf-disease-classification/train_images')
print(len(dataset))
print(dataset[64])

In [ ]:
# split into training and testing sets
train_data,test_data=random_split(dataset, [int(9*len(dataset)/10),int(1*len(dataset)/10)+1 ], generator=torch.Generator().manual_seed(42))

# insert into dataloaders 
# since number of classes are 5 , we will keep batch size 16
train_dataloader=DataLoader(dataset=train_data,batch_size=16,shuffle=True)
test_dataloader=DataLoader(dataset=test_data,batch_size=16,shuffle=True)

In [ ]:
# import models for ensembling
num_models=3 # i plan to do 2 of each type resnet, MobileNet V2,VGG-19 with batch normalization
models_list=[models.shufflenet_v2_x0_5(pretrained=True),models.resnet18(pretrained=True),models.googlenet(pretrained=True)]
models_list=models_list

In [ ]:
# modify last layer
for model in models_list:
    in_ft = model.fc.in_features
    model.fc = nn.Linear(in_ft,5)
    model.to(device)

In [ ]:
# make 9 splits of train_data
training_splits=random_split(train_data, [int(len(train_data)/3) for i in range(2)]+[int(len(train_data)/3)], generator=torch.Generator().manual_seed(42))

In [ ]:
split_dataloaders=[]
for split in training_splits:
    dataloader_split=DataLoader(dataset=split,batch_size=16,shuffle=True)
    split_dataloaders.append(dataloader_split)
print(split_dataloaders)

In [ ]:


def train(epoch,model,dataloader): 
    model.train()
    optimizer = optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.999))
    losss=nn.CrossEntropyLoss()
    viz=[]
    for batch_idx,(data,label) in enumerate(dataloader):
        
        data,label=data.to(device),label.to(device)
        
        optimizer.zero_grad()
        output=model(data)

        # we use nll loss becuase we have added logsoftmax layer , or else we would have used cross entropy which combines both
        loss=losss(output,label)
        viz.append([batch_idx,loss])
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(dataloader.dataset),
                    100. * batch_idx / len(dataloader), loss.item()))
    return viz

def test(model,dataloader):
    model.eval()
    with torch.set_grad_enabled(False):
        size = len(dataloader.dataset)
        pred_correct=0
        for batch_id,(data,label) in enumerate(dataloader):
            data,label=data.to(device),label.to(device)
            output=model(data)
            pred_correct += (label == torch.argmax(output,dim=1)).sum()
  
    print('the accuracy is {}'.format(pred_correct*100/size))
  

In [ ]:
# time to train ensemble
for epoch in range(0,5):
    for i in range(len(models_list)):
        viz=train(epoch,models_list[i],split_dataloaders[i])
        plt.plot([i[0] for i in viz],[i[1] for i in viz])
        plt.show()
        

In [ ]:
def ensemble_test(models_list,test_dataloader):
    size = len(test_dataloader.dataset)
    pred_correct=0
    for model in models_list:
        model.eval()
        
    with torch.set_grad_enabled(False):
        for batch_id,(data,label) in enumerate(test_dataloader):
            onehot=torch.zeros(len(label),5)
            data,label,onehot=data.to(device),label.to(device),onehot.to(device)
            for model in models_list:
                output=model(data)
                #predicted_labels=torch.argmax(output,dim=1).to(device)
                #onehot+=torch.nn.functional.one_hot(predicted_labels,5)
                onehot+= output
            batch_pred=torch.argmax(onehot,dim=1)
            pred_correct += (label == batch_pred).sum()
        print('accuaracy on the test set for model ensemble is {}'.format(pred_correct*100/size))            

In [ ]:
ensemble_test(models_list,test_dataloader)

In [ ]:
class testImageDataset(Dataset): # submission
    def __init__(self, img_dir, transform=Compose([Resize((256,256)),RandomResizedCrop(224),ConvertImageDtype(dtype=torch.float),Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]), target_transform=None):
       # self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.name_list=[]
        for dirname, _, filenames in os.walk(img_dir):
            for filename in filenames:
                self.name_list.append(filename)
        
    def __len__(self):
        return len(self.name_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.name_list[idx])
        image = read_image(img_path)
        #label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        #sample = {"image": image, "label": label}
        return image,self.name_list[idx]


In [ ]:
def ensemble_sub(models_list,test_dataloader):
    size = len(test_dataloader.dataset)
    pred_correct=0
    for model in models_list:
        model.eval()
    df = pd.DataFrame(columns=['image_id','label'])
    with torch.set_grad_enabled(False):
        for batch_id,(data,img_path) in enumerate(test_dataloader):
            onehot=torch.zeros(len(img_path),5)
            data,onehot=data.to(device),onehot.to(device)
            for model in models_list:
                output=model(data)
                #predicted_labels=torch.argmax(output,dim=1).to(device)
                #onehot+=torch.nn.functional.one_hot(predicted_labels,5)
                onehot+= output
            batch_pred=torch.argmax(onehot,dim=1)
            #print(img_path[0])
            for i in range(len(img_path)):
                df=df.append({'image_id':str(img_path[i]),'label':int(batch_pred[i].item())},ignore_index=True)
            #pred_correct += (label == batch_pred).sum()
    return df

In [ ]:
test_sub=testImageDataset('../input/cassava-leaf-disease-classification/test_images')
sub_dataloader=DataLoader(dataset=test_sub,batch_size=16,shuffle=False)
df=ensemble_sub(models_list,sub_dataloader)
df.to_csv('submission.csv',index_label=False)